<a href="https://colab.research.google.com/github/ginttone/summary/blob/master/NLP/RecommendationSystemWithDoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 책 추천 서비스

In [1]:
!ls ./drive/MyDrive/Multi_test_deeplearning/data.csv

./drive/MyDrive/Multi_test_deeplearning/data.csv


In [2]:
import pandas as pd

In [8]:
df_data = pd.read_csv('./drive/MyDrive/Multi_test_deeplearning/data.csv')

In [9]:
df_data.head(4)

,Unnamed: 0,Desc,Unnamed: 0.1,author,genre,image_link,rating,title
0,0,We know that power is shifting: From West to E...,0.0,Moisés Naím,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.63,The End of Power: From Boardrooms to Battlefie...
1,1,Following the success of The Accidental Billio...,1.0,Blake J. Harris,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.94,"Console Wars: Sega, Nintendo, and the Battle t..."
2,2,How to tap the power of social software and ne...,2.0,Chris Brogan,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.78,Trust Agents: Using the Web to Build Influence...
3,3,William J. Bernstein is an American financial ...,3.0,William J. Bernstein,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.20,The Four Pillars of Investing


In [10]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2382 entries, 0 to 2381
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    2382 non-null   int64  
 1   Desc          2382 non-null   object 
 2   Unnamed: 0.1  1185 non-null   float64
 3   author        2382 non-null   object 
 4   genre         2382 non-null   object 
 5   image_link    2382 non-null   object 
 6   rating        2382 non-null   float64
 7   title         2382 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 149.0+ KB


```
 1   Desc          2382 non-null   object 
 7   title         2382 non-null   object
```

데이터 길이 같아 불용어처리 외 전처리 하지 않아도 됨

In [11]:
df_data['Desc']

0       We know that power is shifting: From West to E...
1       Following the success of The Accidental Billio...
2       How to tap the power of social software and ne...
3       William J. Bernstein is an American financial ...
4       Amazing book. And I joined Steve Jobs and many...
                              ...                        
2377    Ralph Roberts, a sus setenta años y tras la mu...
2378    Murder at the Vicarage marks the debut of Agat...
2379    In 1951 John Wyndham published his novel The D...
2380    This now classic book revealed Flannery O'Conn...
2381    Imbued on every page with Frank McCourt's asto...
Name: Desc, Length: 2382, dtype: object

## Tokenize 
map = apply 같은 기능

RegexpTokenizer : [Regular Expression](https://regexr.com/) Tokenizer
내가 지정한 글자나 문자대로 짤라낼 수 있다.

단어의 인접행렬을 알아내기 위해 하는 작업


In [14]:
from nltk.tokenize import RegexpTokenizer

In [27]:
text01 = ' We know that power is shifting: From West to E'
 
def remove_punctuation(text):
  tokenizer = RegexpTokenizer('[a-zA-Z]+')  
  text_list = tokenizer.tokenize(text)
  # print(type(text_list),text_list)
  result = ' '.join(text_list)
  # print(type(result),result)
  return result

In [28]:
 remove_punctuation(text01)

<class 'list'> ['We', 'know', 'that', 'power', 'is', 'shifting', 'From', 'West', 'to', 'E']
<class 'str'> We know that power is shifting From West to E


'We know that power is shifting From West to E'

apply 사용은 이와 같다
```
  for text01 in df_data['Desc']
 
  remove_punctuation(text01)
```

In [ ]:
df_data['cleaned'] = df_data['Desc'].apply(remove_punctuation)

In [33]:
df_data['cleaned'].head(5)

0    We know that power is shifting From West to Ea...
1    Following the success of The Accidental Billio...
2    How to tap the power of social software and ne...
3    William J Bernstein is an American financial t...
4    Amazing book And I joined Steve Jobs and many ...
Name: cleaned, dtype: object

In [34]:
df_data.columns

Index(['Unnamed: 0', 'Desc', 'Unnamed: 0.1', 'author', 'genre', 'image_link',
       'rating', 'title', 'cleaned'],
      dtype='object')

In [35]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2382 entries, 0 to 2381
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    2382 non-null   int64  
 1   Desc          2382 non-null   object 
 2   Unnamed: 0.1  1185 non-null   float64
 3   author        2382 non-null   object 
 4   genre         2382 non-null   object 
 5   image_link    2382 non-null   object 
 6   rating        2382 non-null   float64
 7   title         2382 non-null   object 
 8   cleaned       2382 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 167.6+ KB


## Pre-Trained
매번 문장 들어올때마다 워드투백을 돌릴수 없다. 그래서 생각해 낸 것이 Pre-trained (누군가가 엄청많은 데이터를 Word2Vec으로 만들어둔것 불러다 쓰기)으로 사용한다.




In [36]:
# 구글뉴스 위키피디아 - > Word2Vec 으로 만들어둔 자료

!curl -O https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1570M  100 1570M    0     0  94.3M      0  0:00:16  0:00:16 --:--:-- 95.3M


In [38]:
from gensim.models import Word2Vec

```
word2vec_model = Word2Vec(min_count=2, workers=-1, size=300)
word2vec_model.build_vocab(df_data['cleaned']) # need to modify
word2vec_model.intersect_word2vec_format('GoogleNews-vectors-negative300.bin.gz',lockf=1.0, binary=True)
word2vec_model.train(df_data['cleaned'],total_examples=word2vec_model.corpus_count, epochs=15)
word2vec_model.wv.vocab
```

In [ ]:
corpus = []
for words in df_data['cleaned']:
  corpus.append(words.split())
  print(type(corpus),corpus[4:8])

In [48]:
word2vec_model = Word2Vec(min_count=2, workers=-1, size=300)

In [49]:
word2vec_model.build_vocab(corpus) 

In [50]:
word2vec_model.intersect_word2vec_format('GoogleNews-vectors-negative300.bin.gz',lockf=1.0, binary=True)

In [51]:
word2vec_model.train(corpus,total_examples=word2vec_model.corpus_count, epochs=15)

(0, 0)

각 단어마다 gensim으로 한 Keyedvectors 가 들어간 것을 볼수있다

In [ ]:
word2vec_model.wv.vocab

벡터라이제이션 교육시킨 기준으로 백터화 (수치화) 

300차원 워드투백으로 만들어져있다. 

수치화된 것을 합산(사이즈 같을 경우) 후 평균구함
```
for line in df_data['cleaned'][2:3] : #1개만 
  # print(line)
  doc2vec = 0
  for word in line.split():
    # print(word) 
    if word in word2vec_model.wv.vocab:
      # print(word2vec_model[word]) 워드투백에 저장되있는 숫자가 나옴
      doc2vec = doc2vec + word2vec_model[word] #합산
      
  doc2vec = doc2vec/len(doc2vec)# 평균
  print(type(doc2vec), doc2vec)
```


In [72]:
doc_embedding_list = list()
for line in df_data['cleaned']:
  doc2vec = None
  for word in line.split():
    if word in word2vec_model.wv.vocab:
      if doc2vec is None:
        doc2vec = word2vec_model[word] 
      else :
        doc2vec = doc2vec + word2vec_model[word] 

  if doc2vec is not None:
    doc2vec = doc2vec / len(doc2vec)
    doc_embedding_list.append(doc2vec)

print(doc_embedding_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
type(doc_embedding_list),doc_embedding_list[3]

```
(list,
 array([ 4.85460274e-03,  6.44234102e-03,  2.02899147e-03,  7.35819340e-03,
        -1.05098961e-03, -4.15791292e-03,  4.43558767e-03, -1.21494001e-02,
         1.21053243e-02, -1.25548846e-04, -5.31814434e-03, -7.44484272e-03,
         ...
```

## Cosin_Similarity metrix
인접행렬

In [75]:
from sklearn.metrics.pairwise import cosine_similarity

In [76]:
cosine_sim= cosine_similarity(doc_embedding_list, doc_embedding_list)
type(cosine_sim),cosine_sim.shape

(numpy.ndarray, (2381, 2381))

In [77]:
cosine_sim[3]

array([0.7652141 , 0.7288712 , 0.7382817 , ..., 0.7206474 , 0.67123234,
       0.75328106], dtype=float32)

## Predict

In [78]:
title = 'The Da Vinci Code'
title = 'The Four Pilars of investing' #3

3(4번째) 각각 해당되는 내용을 리스트로 나오게 함

In [ ]:
idx = 3
sim_scores = list(enumerate(cosine_sim[3]))
print(sim_scores)

선택한 3(4번쨰) 책의 근사치 리스트들이 나오게됨

In [82]:
sim_scores_list = sorted(sim_scores,key=lambda x:x[1], reverse=True)

In [85]:
# sim_scores_list[0:6] 검색 3(4번쨰)자신 포함
sim_scores_list[1:6]

[(6, 0.8593486),
 (225, 0.8524401),
 (537, 0.8478668),
 (606, 0.8467616),
 (331, 0.84570736)]

In [86]:
df_data.head(7)

,Unnamed: 0,Desc,Unnamed: 0.1,author,genre,image_link,rating,title,cleaned
0,0,We know that power is shifting: From West to E...,0.0,Moisés Naím,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.63,The End of Power: From Boardrooms to Battlefie...,We know that power is shifting From West to Ea...
1,1,Following the success of The Accidental Billio...,1.0,Blake J. Harris,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.94,"Console Wars: Sega, Nintendo, and the Battle t...",Following the success of The Accidental Billio...
2,2,How to tap the power of social software and ne...,2.0,Chris Brogan,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.78,Trust Agents: Using the Web to Build Influence...,How to tap the power of social software and ne...
3,3,William J. Bernstein is an American financial ...,3.0,William J. Bernstein,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.20,The Four Pillars of Investing,William J Bernstein is an American financial t...
4,4,Amazing book. And I joined Steve Jobs and many...,4.0,Akio Morita,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.05,Made in Japan: Akio Morita and Sony,Amazing book And I joined Steve Jobs and many ...
5,5,In the tradition of Phil Knight's Shoe Dog com...,5.0,Marc Randolph,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.28,That Will Never Work: The Birth of Netflix and...,In the tradition of Phil Knight s Shoe Dog com...
6,6,"Gregory Zuckerman, the bestselling author of T...",6.0,Gregory Zuckerman,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.12,The Man Who Solved the Market: How Jim Simons ...,Gregory Zuckerman the bestselling author of Th...
